In [15]:
# импорт функций поиска в корпусе и вспомогательных библиотек
from search_functions import *
from tqdm import tqdm
import pickle

In [2]:
# загрузка сохраненного списка прилагательных класса ПЛОДЫ
with open('fruits.txt', 'r', encoding='utf-8') as f:
    fruits = f.read().split('\n')[:-1]

In [3]:
# загрузка информации о сочетаемости, сохранение в виде словаря
pattern = {}
with open('compatibility2.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
for line in lines:
    line = line.replace('\n', '')
    sense_num = line.split(' ')[0]
    pattern[sense_num] = line.split(' ')[1:]

In [4]:
# получение информации о числе вхождений в корпус
fruit_freq = {}
for fruit in fruits:
    frequency(fruit, fruit_freq)

In [5]:
# заготовка словаря, хранящего информацию о значениях
senses = {}
for fruit in fruits:
    senses[fruit] = []

In [6]:
# функция поиска значений
def sense_search(word, sense_num):
    i = 0
    while sense_num not in senses[word] and i < len(pattern[sense_num]):
        r = bigramm_search(word, pattern[sense_num][i])
        if re.search('(\d)+( вхожден.{2})', BeautifulSoup(r.text).get_text()) is not None:
            senses[word].append(sense_num)
        i += 1
    return

In [7]:
# поиск значений и сохранение в подготовленный словарь
for fruit in tqdm(fruits):
    if fruit_freq[fruit] != 0:
        for k in pattern.keys():
            sense_search(fruit, k)

100%|████████████████████████████████████████████████████████████████████████████████| 78/78 [1:50:04<00:00, 84.67s/it]


In [8]:
# посмотрим на результаты поиска
for k, v in senses.items():
    print(k, v)

абрикосовый ['1.2', '1.3', '1.4', '1.6', '1.7', '2']
авокадовый []
айвовый ['1.2', '2']
алычовый ['1.2', '2']
ананасный ['1.2', '1.3', '1.4', '1.6', '2']
ананасовый ['1.2', '1.4']
анисовый ['1.2', '1.3', '1.4', '1.5', '1.6']
арахисовый ['1.2', '1.4']
арбузный ['1.2', '1.4']
баклажанный ['1.1']
банановый ['1.2', '1.4', '2']
барбарисовый ['1.2', '1.3', '1.4', '2']
бобовый ['1.2']
боярышниковый []
брусничный ['1.2', '1.3', '1.4', '1.7', '2']
виноградный ['1.1', '1.2', '1.3', '1.4', '1.5', '1.7', '2']
вишневый ['1.1', '1.2', '1.3', '1.4', '1.6', '1.7', '2']
голубичный ['1.2', '2']
гороховый ['1.2', '1.3', '1.4', '1.7']
гранатовый ['1.2', '1.4', '1.7', '2']
грушевый ['1.2', '1.3', '1.6', '2']
дынный ['1.1', '1.2', '1.6', '2']
дюшесный []
ежевичный ['1.2', '1.3', '2']
желудевый ['1.4']
жимолостный ['1.2']
земляничный ['1.1', '1.2', '1.3', '1.4', '1.5', '1.6', '1.7', '2']
зерновой ['1.1', '1.6']
инжирный ['1.2', '2']
кабачковый ['1.2']
калиновый ['1.2', '1.3', '2']
каштановый ['1.6', '1.7', '

In [9]:
# удаляем прилагательные, для которых ничего не нашлось ни автоматически, ни вручную
del senses['авокадовый']
del senses['боярышниковый']
del senses['дюшесный']

In [11]:
# выводим прилагательные, которые нужно проверить на сочетаемость со словом 'цвет'
for fruit in senses.keys():
    if '1.7' not in senses[fruit]:
        r = bigramm_search(fruit, 'цвет')
        if re.search('(\d)+( вхожден.{2})', BeautifulSoup(r.text).get_text()) is not None:
            print(fruit)

анисовый
баклажанный
банановый
бобовый
грушевый
желудевый
калиновый
клубничный
крыжовенный
маковый
маслинный
миндальный
морошковый
огуречный
рябиновый
сливовый
смородиновый
томатный
черешневый
черничный


In [12]:
# добавляем значение 1.7 нужным прилагательным
colors = ['анисовый', 'баклажанный', 'банановый', 'желудевый', 'клубничный', 
          'крыжовенный', 'маковый', 'маслинный', 'миндальный', 'морошковый', 
          'огуречный', 'рябиновый', 'сливовый', 'смородиновый', 'томатный', 'черешневый', 'черничный']
for fruit in colors:
    senses[fruit].append('1.7')

In [13]:
# редкие прилагательные: добавляем значения, найденные вручную
senses['рябинный'].append('1.2')
senses['томатовый'].append('1.7')
senses['шиповниковый'].append('1.2')
senses['айвовый'].append('1.4')
senses['чечевичный'].append('1.2')
senses['шишечный'].extend(['1.1', '1.2'])
senses['жимолостный'].extend(['1.1', '2'])

In [16]:
# сохраняем словарь значений
with open('senses_fruits.pickle', 'wb') as f:
    pickle.dump(senses, f)

In [17]:
# сохраняем словарь частотности
with open('fruits_freq.pickle', 'wb') as f:
    pickle.dump(fruit_freq, f)